In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
#Configure Defaults
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.family']='Malgun Gothic'
import missingno as msno
from tqdm import tqdm_notebook

In [4]:
df = pd.read_pickle('../1399_data/1399_dataset.pkl')
df.head(3)

,내부고발여부,문서번호,소비자 성명,소비자 주소,식품유형,신고내용,신고내용(내용기재) 200자,신고분류,신고소분류,신고일시,...,조사불가,년도,월,요일,시도,시군구,소시도,소시군구,조시도,조시군구
0,N,A12015-0000108455,최은순,경기도 수원시 영통구 효원로 363 매탄 위브 하늘채 102동 1501호,NaN,이물발견,"2014-01-01, `오뚜기옛날자른당면`을 개봉하여 조리를 하기 위해 물에 담가보...",기타,NaN,2015-01-02,...,0,2015.0,1.0,4.0,NaN,NaN,경기도,수원시,경기도,수원시
1,N,A12015-0000108456,이종범,강원도 원주시 흥업면 한촌길 17 101호,NaN,이물발견,12/30 여주마트에서 땅콩을 구매하심.\n1/1 제품을 개봉하여 드심.\n일부가 ...,기타,NaN,2015-01-02,...,0,2015.0,1.0,4.0,NaN,NaN,강원도,원주시,경기도,여주시
2,N,A12015-0000108451,김민규,경기도 화성시 병점동 느치미마을 주공3단지 301동 203호,NaN,이물검출외(축산물),*유통기한경과*\n민원인이 1월 1일 오전 8시 47분경 세븐일레븐 강남2호점에서 ...,기타,NaN,2015-01-02,...,0,2015.0,1.0,4.0,NaN,NaN,경기도,화성시,서울특별시,서초구


In [5]:
df.columns

Index(['내부고발여부', '문서번호', '소비자 성명', '소비자 주소', '식품유형', '신고내용', '신고내용(내용기재) 200자',
       '신고분류', '신고소분류', '신고일시', '이물검출기타', '이물검출명', '이물검출코드', '제조원', '제조원 소재지',
       '제조원업종', '제품구입장소', '제품명', '조사결과(이물보고)', '조사결과(접수원본) 200자', '조사종결내용',
       '종결일시', '진행상태', '처리기간', '행정처분', '현재 처리부서', '시정명령', '과태료', '과징금', '시설개수',
       '경고', '음식품폐기', '제품폐기', '원료폐기', '품목정지', '품목류정지', '품목제조정지', '품목류제조정지',
       '영업정지', '해당차량영업정지', '영업허가취소', '영업취소', '허가취소', '영업등록취소', '등록취소', '업체폐쇄',
       '업소폐쇄', '영업소폐쇄', '폐쇄', '형사고발', '고발', '적발조치', '진행', '진행중', '취하', '이첩',
       '조사불가', '년도', '월', '요일', '시도', '시군구', '소시도', '소시군구', '조시도', '조시군구'],
      dtype='object')

# <font color='blue'> 1.  검색어(57개)로 수집정보의 제목과 내용 count

In [25]:
interest_keywors = pd.read_excel("../1399_data/관심주제어.xlsx")
interest_keywors['주제어'] = interest_keywors['주제어'].apply(lambda x : str(x).strip()) # ' 껌' 공백제거
interest_keywors = interest_keywors['주제어']
len(interest_keywors)
interest_keywors.head()

332

0     상한
1     변질
2     썩은
3    곰팡이
4     벌레
Name: 주제어, dtype: object

# <font color='blue'> 2.  관심주제어(332개, key_words)를 주제어로 TF-IDF 생성
## 2.1 DF 만들기

In [26]:
key_words = pd.read_excel("../1399_data/관심주제어.xlsx")
key_words['주제어'] = key_words['주제어'].apply(lambda x : str(x).strip()) # ' 껌' 공백제거
key_words = key_words['주제어']

','.join(key_words)
len(key_words)
len(set(key_words))

'상한,변질,썩은,곰팡이,벌레,구더기,부패,불량,냄새,악취,유해,농약,독성,항생제,발암물질,방부제,색소,동물용의약품,곰팡이독소,유독물질,패독,버섯독,마약류,GMO,유전자조작,비식용,불법첨가제,멜라민,색소,사용금지,미승인,의약품,비식품,유통기한경과,기생충,불법도축,밀도축,병든,밀도살,불법도살,불량고기,질병감염동물,병사동물,기생충,포장,용기,가소재,중금속용출,증발잔류물,가소제용출,총용출량,원료성분용출,잔류규격위반,기준치초과,부적합,이물혼입,이물질,위반,첨가물,잔류농약,동물용의약품,방사능,중금속,벤조피렌,다이옥신,곰팡이,벌레,금속,유리,플라스틱,성분검사,이미,이취,이물감,재사용,기준치초과,위생불량,비위생,대장균,세균수,위생제조기준위반,반찬 재사용,강아지, 개, 고양이, 애완동물,밀수입,밀반입,수입신고,가짜,짝퉁,중량미달,불법증량,함량미달,거짓,카라멜,불량품질,불향성분,상표도용,원료건전성불량,저급원료혼합,유사성분혼입,품질위조,허위표시,영양,영양표시,미표시,식중독,대장균,장염,바이러스,살모넬라,병원성균,식중독,규격미달,기준미달,부적합 성분,성분함량 규격위반,품질규격위반,무허가,무신고,원산지허위,원산지 속인,원산지 미표시,원산지 미표기,유통기한,유통기한 미표시,유통기한 위변조,유통기한 경과제품,허위과장,과대광고,소비자혼동,허위과대광고,어린이,정서저해,미끼,피해,설사,식중독,알레르기,병원,입원,구토,두드러기,복통,배탈,내과,치과,응급실,진료,약국,음료,캔,콜라,병,뚜껑,커피,차,탄산,과.채주스,주스,과.채음료,두유,건강관련식품,건강,다이어트,건강기능식품,비타민,효능,효과,크릴오일,새싹보리,노니,즉석식품,도시락,햇반,데워먹는,레토르트,전자레인지,축산물,스팸,곰탕,고기,우유,치즈,영아용조제유,성장기용조제유,햄버거,버거,패티,맥도날드,롯데리아,맘스터치,버거킹,간식,과자,사탕,껌,빵,케이크,면류,라면,컵라면,라면봉지,유탕면,건면,숙면,생면,인터넷,모바일,온라인,카페,블로그,인스타,해외직구,앱,어플,몰,홈쇼핑,인터넷,배송,해외사업자,인터넷사이트,G마켓,11번가,쿠팡,위메프,티몬,인

332

313

In [27]:
df.columns

Index(['내부고발여부', '문서번호', '소비자 성명', '소비자 주소', '식품유형', '신고내용', '신고내용(내용기재) 200자',
       '신고분류', '신고소분류', '신고일시', '이물검출기타', '이물검출명', '이물검출코드', '제조원', '제조원 소재지',
       '제조원업종', '제품구입장소', '제품명', '조사결과(이물보고)', '조사결과(접수원본) 200자', '조사종결내용',
       '종결일시', '진행상태', '처리기간', '행정처분', '현재 처리부서', '시정명령', '과태료', '과징금', '시설개수',
       '경고', '음식품폐기', '제품폐기', '원료폐기', '품목정지', '품목류정지', '품목제조정지', '품목류제조정지',
       '영업정지', '해당차량영업정지', '영업허가취소', '영업취소', '허가취소', '영업등록취소', '등록취소', '업체폐쇄',
       '업소폐쇄', '영업소폐쇄', '폐쇄', '형사고발', '고발', '적발조치', '진행', '진행중', '취하', '이첩',
       '조사불가', '년도', '월', '요일', '시도', '시군구', '소시도', '소시군구', '조시도', '조시군구'],
      dtype='object')

In [28]:
food_info = df[['신고일시', '신고내용(내용기재) 200자']]
food_info.columns =['신고일시', '내용']

In [29]:
food_info['내용'] = food_info['내용'].fillna('조상')

In [30]:
food_info.index = np.arange(0, len(food_info)) # To avoid (can not assign duplicate index)

In [31]:
def food_type (search_text) :
    for i, text in enumerate(food_info['내용']) :
        if search_text in regex.findall(text) :
            food_info.loc[i,search_text] = 1 #len(regex.findall(text)) # 단어출현빈도인 len(regex.findall(text)) 대신 '1'

In [32]:
import re

for search_text in tqdm_notebook(key_words) :
    regex=re.compile(search_text)
    food_info[search_text] = 0
    food_type (search_text)

In [35]:
food_info

,신고일시,내용,상한,변질,썩은,곰팡이,벌레,구더기,부패,불량,...,품목제조정지,시정명령,적합,회수,폐기,판매중단,시험검사,반복,오인,차단
0,2015-01-02 00:00:00,"2014-01-01, `오뚜기옛날자른당면`을 개봉하여 조리를 하기 위해 물에 담가보...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-02 00:00:00,12/30 여주마트에서 땅콩을 구매하심.\n1/1 제품을 개봉하여 드심.\n일부가 ...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-02 00:00:00,*유통기한경과*\n민원인이 1월 1일 오전 8시 47분경 세븐일레븐 강남2호점에서 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-02 00:00:00,"오늘 민원인께서 공군회관 맞은편 씨유편의점에서 `롯데 아몬드 빼빼로`를 구입하였고,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-02 00:00:00,12/31 소담(수입식품가게)에서 무장아찌구매하심.\n유통기한이 13.02.28까지...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52879,2019-12-31 17:26:00,<세븐일레븐 응암푸르지오점 유통기한 경과 제품 판매>\n민원인은 2019-12-30...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52880,2019-12-31 11:28:00,<코랄칼슘마그네슘플러스비타민디 가품 의심> 한글 파일 첨부,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52881,2019-12-31 10:27:00,<노브랜드맛있는 군밤 곰팡이 이물 발견>\n민원인은 2019-12-11 노브랜드 마...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52882,2019-12-31 09:16:00,"<브라질넛 위해성의심> \n민원인은 2019-12월 중순경, 이마트 트레이더스 구성...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
## str.count()와 findall() 같은 결과

In [37]:
from ckonlpy.tag import Twitter
twitter = Twitter()

In [42]:
search_text ='벌레'

In [48]:
food_info['내용'][0]
twitter.nouns(food_info['내용'][3]).count(search_text)

'2014-01-01, `오뚜기옛날자른당면`을 개봉하여 조리를 하기 위해 물에 담가보니, 10마리정도 되는 벌레가 떠올랐다고 함. 업체에도 말씀을 하시고, 구입처 점장과도 제품과 이물을 확인한 상태. 해당 제품에서 이러한 이물이 발견된 부분에 대해 문제가 있다고 판단되어 1399로 신고접수. 해당 제품과 이물에 대한 원인조사 요청. \n\n*정확한 구입일은 확인불가. 한달이내라고 말씀하심.'

0

In [50]:
regex=re.compile(search_text)
regex.findall(food_info['내용'][3])

[]

In [31]:
def food_type (search_text) :
    for i, text in enumerate(food_info['내용'][0:10]) :
        if search_text in regex.findall(text) :
            food_info.loc[i,search_text] = 1 #len(regex.findall(text)) # 단어출현빈도인 len(regex.findall(text)) 대신 '1'

In [ ]:
for search_text in tqdm_notebook(key_words) :
    regex=re.compile(search_text)
    food_info[search_text] = 0
    food_type (search_text)

In [34]:
food_info.columns

Index(['신고일시', '내용', '상한', '변질', '썩은', '곰팡이', '벌레', '구더기', '부패', '불량',
       ...
       '품목제조정지', '시정명령', '적합', '회수', '폐기', '판매중단', '시험검사', '반복', '오인', '차단'],
      dtype='object', length=315)

In [20]:
food_info.to_pickle('../1399_output/dataset_keywords.pkl')

In [6]:
df = pd.read_pickle('../1399_output/dataset_keywords.pkl')

In [7]:
df.head()

,신고일시,내용,상한,변질,썩은,곰팡이,벌레,구더기,부패,불량,...,품목제조정지,시정명령,적합,회수,폐기,판매중단,시험검사,반복,오인,차단
0,2015-01-02,"2014-01-01, `오뚜기옛날자른당면`을 개봉하여 조리를 하기 위해 물에 담가보...",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-02,12/30 여주마트에서 땅콩을 구매하심.\n1/1 제품을 개봉하여 드심.\n일부가 ...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-02,*유통기한경과*\n민원인이 1월 1일 오전 8시 47분경 세븐일레븐 강남2호점에서 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-02,"오늘 민원인께서 공군회관 맞은편 씨유편의점에서 `롯데 아몬드 빼빼로`를 구입하였고,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-02,12/31 소담(수입식품가게)에서 무장아찌구매하심.\n유통기한이 13.02.28까지...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
